<a href="https://colab.research.google.com/github/yhc-666/hate-speech-detection/blob/master/Final_submmision_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [27]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Aug  5 12:37:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
df_dataset = pd.read_csv('/content/drive/MyDrive/50.007kaggle/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/50.007kaggle/test.csv')
df_dataset_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/train_tfidf_features.csv")
df_test_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/test_tfidf_features.csv")

In [33]:
# y of whole dataset
labels = df_dataset_features.iloc[:,[1]]
# X of whole dataset
features = df_dataset_features.iloc[:,2:]

# split train test set
X_train, X_validation, y_train, y_validation = train_test_split(features, labels, test_size=0.05, random_state = 4)

# Model1: Random Forest (Best model used for final submission)

Note: we first use 10-fold cross validation to find best hyper parameters to determine the model, then train the determined model on whole dataset. So we are not showing prediction/performance on validation set. Detail of hyperparameter tuning will be discussed in Task 4 report 

In [43]:
model_rf = RandomForestClassifier(n_estimators=131, 
                               min_samples_split = 5,
                               class_weight = {0:1, 1:2},
                               random_state = 4)

In [26]:
model_rf.fit(features,labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(class_weight={0: 1, 1: 2}, min_samples_split=5,
                       n_estimators=131, random_state=4)

# Model2: LDA

In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression

Train a LDA model on X_train and use it to transform X_train to X_train_LDA 

In [18]:
sklearn_lda = LDA(n_components=1).fit(X_train,y_train)
X_train_lda = sklearn_lda.transform(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train a logistic regression model on transformed X_train

In [19]:
lr = LogisticRegression()
lr.fit(X_train_lda,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

Perfrom LDA transform using trained LDA model on X_validation to get X_validation_LDA, and perform prediction using X_validation_LDA and trained logistic regression model.

In [20]:
X_validation_lda = sklearn_lda.transform(X_validation)

In [21]:
y_validation, y_pred_validation = y_validation, lr.predict(X_validation_lda)
threshold = 0.5
for i in range(len(y_pred_validation)):
    y_pred_validation[i] = 1 if y_pred_validation[i] > threshold else 0

Evaluate our model prediction on validation set using macro F1 score. 

In [23]:
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  

0.5472972972972973
0.7624113475177304
validation_macro_f1 : 0.6549


# Model3: LightGBM

In [24]:
import lightgbm as lgb

Set hyperparameters of model

In [28]:
model_lgbm = lgb.LGBMClassifier(boosting_type = 'gbdt',  # 设置提升类型
    objective = 'binary', 
    num_iterations = 1000,
    max_depth = -1,
    metric = 'macro_f1',  
    num_leaves = 220,  
    learning_rate = 0.01,  
    feature_fraction = 0.9,  
    bagging_fraction = 0.65,  
    bagging_freq =  5,  
    min_data_in_leaf = 20,
    scale_pos_weight = 1.8,
    lambda_l2 = 5,
    nthread = -1,)

Train model on training set

In [34]:
model_lgbm.fit(X_train,y_train, eval_metric = 'f1_macro')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(bagging_fraction=0.65, bagging_freq=5, feature_fraction=0.9,
               lambda_l2=5, learning_rate=0.01, metric='macro_f1',
               min_data_in_leaf=20, nthread=-1, num_iterations=1000,
               num_leaves=220, objective='binary', scale_pos_weight=1.8)

Use model to make prediction on validation_set

In [35]:
y_validation, y_pred_validation = y_validation, model_lgbm.predict(X_validation)

threshold = 0.5
for i in range(len(y_pred_validation)):
    y_pred_validation[i] = 1 if y_pred_validation[i] > threshold else 0

Evaluate our model prediction on validation set using macro F1 score.

In [36]:
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  

0.6513629842180775
0.7624633431085045
validation_macro_f1 : 0.7069


# Model4: XGboost Tree 

In [37]:
from xgboost import XGBClassifier

Set hyperparameters of model

In [41]:
model_XGBT = XGBClassifier(learning_rate=0.01,
                      n_estimators=100,          
                      max_depth=10,               
                      min_child_weight = 1,     
                      gamma=0.,                  
                      reg_lambda = 1.2,
                      subsample=0.6,             
                      colsample_btree=1,         
                      scale_pos_weight=2,        
                      random_state=1,           
                      slient = 0
                      )

Train model on training set

In [42]:
model_XGBT.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(colsample_btree=1, gamma=0.0, learning_rate=0.01, max_depth=10,
              random_state=1, reg_lambda=1.2, scale_pos_weight=2, slient=0,
              subsample=0.6)

Use model to make prediction on validation_set

In [44]:
y_validation, y_pred_validation = y_validation, model_XGBT.predict(X_validation)

Evaluate our model prediction on validation set using macro F1 score.

In [45]:
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  

0.5967302452316077
0.6997971602434078
validation_macro_f1 : 0.6483


# Model5: Logistic regression

Train model on training set

In [49]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

Use model to make prediction on validation_set

In [50]:
y_validation, y_pred_validation = y_validation, lr.predict(X_validation)
threshold = 0.5
for i in range(len(y_pred_validation)):
    y_pred_validation[i] = 1 if y_pred_validation[i] > threshold else 0

Evaluate our model prediction on validation set using macro F1 score.

In [51]:
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  

0.5601436265709157
0.7893379191745485
validation_macro_f1 : 0.6747


# Use our best model(random forest) to predict on test set and save prediction in a csv file

In [10]:
X_test = df_test_features.iloc[:,1:]

In [12]:
y_test = model_rf.predict(X_test)

In [13]:
print(y_test)

[1 0 1 ... 1 0 1]


In [14]:
x = df_test
x = x.set_index("id")
x = x.drop("post", axis = 1)
x["label"] = y_test
print(x)
x.to_csv("/content/drive/MyDrive/50.007kaggle/submit.csv")

       label
id          
17185      1
17186      0
17187      1
17188      0
17189      0
...      ...
21476      1
21477      1
21478      1
21479      0
21480      1

[4296 rows x 1 columns]
